In [6]:
import os

# Cartelle
input_folder = "./JSON/standard"
output_folder = "./JSON/standard_cleaned"

os.makedirs(output_folder, exist_ok=True)


def is_empty(line):
    return line.strip() == ""


for filename in os.listdir(input_folder):

    input_path = os.path.join(input_folder, filename)

    if not os.path.isfile(input_path):
        continue

    with open(input_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    cleaned = []

    i = 0
    n = len(lines)

    while i < n:

        # Controlla se inizia una sequenza vuota-testo-vuota
        if (
            is_empty(lines[i]) and
            i + 2 < n and
            not is_empty(lines[i + 1]) and
            is_empty(lines[i + 2])
        ):

            # Salta tutto il blocco
            i += 1

            while i < n:

                # Salta una riga di testo
                if i < n and not is_empty(lines[i]):
                    i += 1

                # Salta una riga vuota
                if i < n and is_empty(lines[i]):
                    i += 1
                else:
                    break

        else:
            cleaned.append(lines[i])
            i += 1

    output_path = os.path.join(output_folder, filename)

    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(cleaned)

    print(f"Processato: {filename}")

print("Tutti i file completati.")


Processato: solutionCLEX1.json
Processato: solutionCLEX2.json
Processato: solutionCLEX3.json
Processato: solutionCLEX4.json
Processato: solutionCLEX5.json
Processato: solutionCLEX6.json
Processato: solutionHeu1.json
Processato: solutionHeu2.json
Processato: solutionHeu3.json
Processato: solutionHeu4.json
Processato: solutionHeu5.json
Processato: solutionHeu6.json
Tutti i file completati.


In [2]:
import json
import math

def mean(values):
    return sum(values) / len(values) if values else 0.0

def std_dev(values):
    filtered = [v for v in values if v != 0]
    if len(filtered) <= 1:
        return 0.0
    m = mean(filtered)
    variance = sum((x - m) ** 2 for x in filtered) / len(filtered)
    return math.sqrt(variance)

def analyze_solution(solution):
    sprint_risks = []
    sprint_uncertainties = []
    sprint_capacity_usage = []

    for sprint in solution.get("sprints", []):
        stories = sprint.get("stories", [])
        if not stories:
            continue  # solo sprint occupati

        total_risk = sum(story.get("risk", 0) for story in stories)
        total_uncertainty = sum(story.get("uncertainty", 0) for story in stories)

        capacity = sprint.get("capacity", 0)
        used_capacity = sprint.get("used_capacity", 0)

        if capacity > 0:
            sprint_capacity_usage.append(used_capacity / capacity)

        sprint_risks.append(total_risk)
        sprint_uncertainties.append(total_uncertainty)

    return {
        "solution_name": solution.get("name", "unknown"),
        "num_sprint_considered": len(sprint_risks),

        # Rischio
        "risk_std_dev": std_dev(sprint_risks),
        "risk_max": max(sprint_risks) if sprint_risks else 0.0,
        "risk_min": min(sprint_risks) if sprint_risks else 0.0,

        # Incertezza
        "uncertainty_std_dev": std_dev(sprint_uncertainties),
        "uncertainty_max": max(sprint_uncertainties) if sprint_uncertainties else 0.0,
        "uncertainty_min": min(sprint_uncertainties) if sprint_uncertainties else 0.0,

        # Capacità
        "avg_capacity_usage": mean(sprint_capacity_usage)
    }


In [4]:
import glob
import json
import os


json_files = glob.glob("./JSON/standard_cleaned/*.json")

print("=== GENERAZIONE RESOCONTI ===\n")


for file in json_files:

    # Nome file senza estensione
    base_name = os.path.splitext(file)[0]

    # Nome output personalizzato
    output_file = f"{base_name}_resoconto.txt"

    print(f"Elaboro: {file} → {output_file}")

    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_file, "w", encoding="utf-8") as f_out:

        f_out.write("=== RESOCONTO PER SOLUZIONE ===\n\n")

        for solution in data.get("solutions", []):

            r = analyze_solution(solution)

            line = (
                f" {r['solution_name']} & "
                f" {r['num_sprint_considered']} & "
                f" {r['risk_std_dev']:.3f} & "
                f" {r['risk_max']:.3f} & {r['risk_min']:.3f} & "
                f" {r['uncertainty_std_dev']:.3f} & "
                f" {r['uncertainty_max']:.3f} & {r['uncertainty_min']:.3f} & "
                f" {r['avg_capacity_usage']:.2%} \\\\"
            )

            print(line)
            f_out.write(line + "\n")


print("\nTutti i resoconti sono stati creati.")


=== GENERAZIONE RESOCONTI ===

Elaboro: ./JSON/standard_cleaned\solutionCLEX1.json → ./JSON/standard_cleaned\solutionCLEX1_resoconto.txt
 stance_06_01.txt &  5 &  2.227 &  10.600 & 5.100 &  2.133 &  10.100 & 4.300 &  95.59% \\
 stance_06_02.txt &  7 &  2.345 &  9.200 & 1.800 &  1.993 &  7.700 & 1.600 &  84.68% \\
 stance_06_03.txt &  6 &  3.583 &  13.500 & 1.900 &  4.298 &  15.700 & 1.700 &  88.89% \\
 stance_06_04.txt &  5 &  3.061 &  11.200 & 3.000 &  3.038 &  11.100 & 3.400 &  86.65% \\
 stance_06_05.txt &  5 &  2.034 &  10.100 & 4.000 &  2.070 &  9.800 & 3.800 &  98.34% \\
  &  23 &  1.137 &  6.400 & 2.500 &  0.945 &  5.900 & 2.700 &  95.94% \\
  &  21 &  1.532 &  7.600 & 1.600 &  1.147 &  6.100 & 1.200 &  93.09% \\
  &  21 &  1.462 &  6.500 & 1.200 &  1.104 &  5.800 & 1.700 &  95.87% \\
  &  23 &  1.324 &  7.800 & 2.400 &  1.102 &  7.500 & 2.500 &  96.12% \\
  &  21 &  1.333 &  6.800 & 1.800 &  1.193 &  6.400 & 1.900 &  95.91% \\
  &  21 &  4.165 &  20.000 & 2.700 &  3.031 &  15.5